In [1]:
MYDIR = "/Users/joregan/Playing/rd_ctm_edit"

In [2]:
from pathlib import Path

In [3]:
WD = Path(MYDIR)

In [4]:
from sync_asr.ctm_edit import ctm_from_file

In [10]:
from sync_asr.riksdag.corrections import _CORRECTIONS

In [11]:
def get_corrections():
    corrections = {}
    for line in _CORRECTIONS.split("\n"):
        if line == "":
            continue
        parts = line.split()
        if not parts[0] in corrections:
            corrections[parts[0]] = []
        corrections[parts[0]].append(parts[1])
    return corrections

In [12]:
corrections = get_corrections()

In [27]:
from string import punctuation

def clean_text(work_ref):
    PUNCT = set(punctuation)
    i = 0
    l = len(work_ref)
    while i < l and work_ref[i] in PUNCT:
        i += 1
    j = -1
    while j >= -l and work_ref[j] in PUNCT:
        j -= 1
    return work_ref[i:j+1]

def compare_text(a, b, lc=False):
    word = clean_text(b)
    if a == word:
        return True
    if lc and a == word.lower():
        return True
    return False


In [51]:
PREFIXES = []
with open("prefixes.tsv") as f:
    for line in f.readlines():
        parts = line.strip().split()
        PREFIXES.append((parts[0], f"{parts[1]}-"))
        if not parts[0] in corrections:
            corrections[parts[0]] = []
        corrections[parts[0]].append(parts[1])

In [22]:
a = "abebégymnasiet"
b = "ABB-gymnasiet"
p_a = "abebé"
p_b = "ABB-"
a.startswith(p_a) and b.startswith(p_b)
a[len(p_a):] == b[len(p_b):]

True

In [52]:
PREFIXES

[('aai', 'AI-'),
 ('aändte', 'ANDT-'),
 ('äateamess', 'ERTMS-'),
 ('abe', 'A/B-'),
 ('abebé', 'ABB-'),
 ('abieff', 'ABF-'),
 ('abite', 'ABT-'),
 ('acka', 'AKKA-'),
 ('adeärr', 'ADR-'),
 ('adeessel', 'ADSL-'),
 ('adeessell', 'ADSL-'),
 ('adessa', 'ADSL-'),
 ('adiell', 'ADL-'),
 ('adiesse', 'ADSL-'),
 ('adiessel', 'ADSL-'),
 ('aef', 'AF-'),
 ('aeles', 'ALS-'),
 ('aenddees', 'ANDTS-'),
 ('aendet', 'ANDT-'),
 ('aendete', 'ANDT-'),
 ('aendete', 'ANDT-'),
 ('aendetees', 'ANDTS-'),
 ('aendeteess', 'ANDTS-'),
 ('aendetes', 'ANDTS-'),
 ('aendetess', 'ANDTS-'),
 ('aendite', 'ANDT-'),
 ('aendt', 'ANDT-'),
 ('aendte', 'ANDT-'),
 ('aendteess', 'ANDTS-'),
 ('aendtes', 'ANDTS-'),
 ('aendtss', 'ANDTS-'),
 ('aenteré', 'ANTD-'),
 ('aentidé', 'ANDT-'),
 ('aett', 'A1-'),
 ('äffän', 'FN-'),
 ('äffe', 'FÖ-'),
 ('äffen', 'FN-'),
 ('äffenvi', 'FMV-'),
 ('äfkas', 'FCAS-'),
 ('ai', 'AI-'),
 ('aisaf', 'ISAF-'),
 ('aj', 'AI-'),
 ('aje', 'AI-'),
 ('akå', 'AK-'),
 ('ako', 'AK-'),
 ('akope', 'AKP-'),
 ('akou', 'AKU-

In [23]:
def checker(a, b):
    if compare_text(a, b, True):
        return True
    word = clean_text(b)
    if a in corrections and word in corrections[a]:
        return True
    if a in corrections and word.lower() in corrections[a]:
        return True
    for pfx in PREFIXES:
        if a.startswith(pfx[0]) and b.startswith(pfx[1]):
            return a[len(pfx[0]):] == b[len(pfx[1]):]
    return False

In [33]:
for file in WD.glob("H201UbU15*"):
    lines = ctm_from_file(file)
    for line in lines:
        line.mark_correct_from_function(checker)

In [34]:
with open("/tmp/foozle", "w") as f:
    for line in lines:
        f.write(str(line) + "\n")

In [37]:
pfxes = []
with open("/tmp/list") as f:
    for line in f.readlines():
        parts = line.strip().split(" ")
        a = parts[4]
        b = parts[6]
        if "-" in b:
            bits = b.split("-")
            if len(bits) == 2:
                if a.endswith(bits[1]):
                    pfxes.append((a[:-len(bits[1])], bits[0]))

In [39]:
pfxes = set(pfxes)

In [41]:
pfxes = [x for x in pfxes if x != ""]

In [45]:
with open("/tmp/pfxraw", "w") as f:
    for pfx in pfxes:
        f.write(f"{pfx[0]}\t{pfx[1]}\n")

In [46]:
LINES = """
2442207130019430621 1 327.74 0.459 Ytterst 1.0 Ytterst cor
2442207130019430621 1 328.3 0.3 handlar 1.0 handlar cor
2442207130019430621 1 328.66 0.06 den 1.0 den cor
2442207130019430621 1 328.78 0.08 här 1.0 här cor
2442207130019430621 1 328.92 0.379 frågan 1.0 frågan cor
2442207130019430621 1 330.32 0.079 om 1.0 om cor
2442207130019430621 1 330.5 0.1 att 1.0 att cor
2442207130019430621 1 330.66 0.5 regeringen 1.0 regeringen cor
2442207130019430621 1 331.38 0.42 inte 1.0 inte cor
2442207130019430621 1 331.9 0.18 har 1.0 har cor
2442207130019430621 1 332.2 0.699 parlamentariskt 1.0 parlamentariskt cor
2442207130019430621 1 332.94 0.339 stöd 1.0 stöd cor
2442207130019430621 1 334.06 0.1 för 1.0 för cor
2442207130019430621 1 334.2 0.18 sitt 1.0 sitt cor
2442207130019430621 1 334.42 0.419 förslag 1.0 förslag cor
2442207130019430621 1 334.94 0.12 att 1.0 att cor
2442207130019430621 1 335.1 0.299 låna 1.0 låna cor
2442207130019430621 1 335.64 0.139 till 1.0 till cor
2442207130019430621 1 335.82 0.879 skattesänkningar 1.0 skattesänkningar cor
2442207130019430621 1 337.16 0.079 för 1.0 för cor
2442207130019430621 1 337.3 0.099 dem 1.0 dem cor
2442207130019430621 1 337.5 0.079 som 1.0 som cor
2442207130019430621 1 337.7 0.319 tjänar 1.0 tjänar cor
2442207130019430621 1 338.14 0.3 mest 1.0 mest cor
2442207130019430621 1 338.58 0.02 i 1.0 i cor
2442207130019430621 1 338.66 0.139 vårt 1.0 vårt cor
2442207130019430621 1 338.84 0.16 land 1.0 <eps> ins
2442207130019430621 1 339.84 0.08 jag 1.0 <eps> ins
2442207130019430621 1 340.0 0.259 yrkar 1.0 land. sub
"""

In [47]:
from sync_asr.ctm_edit import CTMEditLine, shift_epsilons, ctm_from_file
lines = [CTMEditLine(from_line=x) for x in LINES.split("\n") if x != ""]

In [50]:
from sync_asr.riksdag.riksdag_align import rd_equals

shift_epsilons(lines, rd_equals)

[2442207130019430621 ((327740, 328199)) Ytterst|Ytterst,
 2442207130019430621 ((328300, 328600)) handlar|handlar,
 2442207130019430621 ((328660, 328720)) den|den,
 2442207130019430621 ((328780, 328860)) här|här,
 2442207130019430621 ((328920, 329299)) frågan|frågan,
 2442207130019430621 ((330320, 330399)) om|om,
 2442207130019430621 ((330500, 330600)) att|att,
 2442207130019430621 ((330660, 331160)) regeringen|regeringen,
 2442207130019430621 ((331380, 331800)) inte|inte,
 2442207130019430621 ((331900, 332080)) har|har,
 2442207130019430621 ((332200, 332899)) parlamentariskt|parlamentariskt,
 2442207130019430621 ((332940, 333279)) stöd|stöd,
 2442207130019430621 ((334060, 334160)) för|för,
 2442207130019430621 ((334200, 334380)) sitt|sitt,
 2442207130019430621 ((334420, 334839)) förslag|förslag,
 2442207130019430621 ((334940, 335060)) att|att,
 2442207130019430621 ((335100, 335399)) låna|låna,
 2442207130019430621 ((335640, 335779)) till|till,
 2442207130019430621 ((335820, 336699)) sk